In [3]:
import os
print(os.getcwd())

/workspace/Haea/models/VariableAnalyzer/tests


In [4]:
import torch
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from absl import app
from absl import flags
from pytorch_lightning.utilities.model_summary import ModelSummary


import sys,os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.VariableAnalyzer.datasets.dataset import CustomDataset
from models.VariableAnalyzer.models.model import VariableAnalyzer
from models.VariableAnalyzer.training.configs import TrainingRunConfig
from models.VariableAnalyzer.training.lightning import TrainModule
from models.VariableAnalyzer.training.callbacks import VariableVaildationCallback

In [6]:
def get_dataset(year_offset: int, time_len: int):
    device = ("cuda" if torch.cuda.is_available() else "cpu" )
    device = torch.device(device)

    weather = WeatherDataset(year_offset, device=device)
    # dataset.shape:  torch.Size([7309, 100, 1450])
    input, target, mean_std, dims = weather.load()
    dataset = CustomDataset(input, target, time_len)
    return (weather.HAS_LEVEL_VARIABLE, weather.NONE_LEVEL_VARIABLE, weather.PRESSURE_LEVELS), dataset, input.shape, mean_std, dims

time_len = 4 * 3
dataset_info, dataset, shape, mean_std, dims = get_dataset(0, time_len)

데이터셋 불러오는 중...
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, latitude: 28, longitude: 28,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float32 39.0 38.75 ... 32.5 32.25
  * level                         (level) int64 50 100 15

Processing futures: 0it [00:00, ?it/s]

Processing futures: 14it [00:03,  4.63it/s]


4.13900 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, longitude: 14, latitude: 33,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 21.0 22.5 ... 67.5 69.0
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 121.5 123.0 ... 141.0
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, longitude, latitude) float32 ...
    10m_v_component_of_wind       (time, longitude, latitude) float32 ...
    2m_temperature                (time, longitude, latitude) float32 ...
    geopotential                  (time, level, longitude, latitude) float32 ...
    geopotential_at_surface       (longitude, latitude) float32 ...
    mean_sea_level_pressure       (time, longitude, latitude) float32 ...
    ...                     

Processing futures: 14it [00:01,  7.65it/s]


2.48515 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, longitude: 17, latitude: 12,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, longitude, latitude) float32 ...
    10m_v_component_of_wind       (time, longitude, latitude) float32 ...
    2m_temperature                (time, longitude, latitude) float32 ...
    geopotential                  (time, level, longitude, latitude) float32 ...
    geopotential_at_surface       (longitude, latitude) float32 ...
    mean_sea_level_pressure       (time, longitude, latitude) float32 ...
    ...             

Processing futures: 14it [00:00, 16.69it/s]


1.25184 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([14613, 86, 1450])


In [9]:
import yaml
import logging
import os

config_path = os.path.join(os.path.dirname(os.getcwd()), 'configs/train_config.yaml')

try:
    with open(config_path) as f:
            config_dict = yaml.safe_load(f)
            config: TrainingRunConfig = TrainingRunConfig.parse_obj(config_dict)
except FileNotFoundError:
        logging.error(f"Config file {config_path} does not exist. Exiting.")
except yaml.YAMLError:
        logging.error(f"Config file {config_path} is not valid YAML. Exiting.")

model = VariableAnalyzer(
            var_len=shape[1],
            time_len=time_len,
            dim_model=shape[2],
            predict_dim=dims[0],
            batch_size=config.training.batch_size,
            num_heads=config.model.num_heads,
            n_encoder_layers=config.model.n_encoder_layers,
            n_decoder_layers=config.model.n_decoder_layers,
            dropout=config.model.dropout
        )

checkpoint = torch.load('/workspace/Haea/models/VariableAnalyzer/tb_logs/lightning_logs/bnc1ycog/checkpoints/epoch=19-step=25540.ckpt')

model.load_state_dict(checkpoint["state_dict"])

RuntimeError: Error(s) in loading state_dict for VariableAnalyzer:
	Missing key(s) in state_dict: "transformer.encoder.layers.0.self_attn.in_proj_weight", "transformer.encoder.layers.0.self_attn.in_proj_bias", "transformer.encoder.layers.0.self_attn.out_proj.weight", "transformer.encoder.layers.0.self_attn.out_proj.bias", "transformer.encoder.layers.0.linear1.weight", "transformer.encoder.layers.0.linear1.bias", "transformer.encoder.layers.0.linear2.weight", "transformer.encoder.layers.0.linear2.bias", "transformer.encoder.layers.0.norm1.weight", "transformer.encoder.layers.0.norm1.bias", "transformer.encoder.layers.0.norm2.weight", "transformer.encoder.layers.0.norm2.bias", "transformer.encoder.layers.1.self_attn.in_proj_weight", "transformer.encoder.layers.1.self_attn.in_proj_bias", "transformer.encoder.layers.1.self_attn.out_proj.weight", "transformer.encoder.layers.1.self_attn.out_proj.bias", "transformer.encoder.layers.1.linear1.weight", "transformer.encoder.layers.1.linear1.bias"